## Laboratorio 3 Interrupciones - Sistemas Operativos
#### Por: Ricardo Mendoza

#### 1. Utilización de un sistema operativo de direccionamiento de 16 bits

Para este Laboratorio se hara uso de una maquina virtual de un sistema de direccionamiento de 16 bits ( tamaño de registros de 32 bits). Dicha maquina virtual contiene el compilar Turbo C version 3 la cual utilizaremos para los siguientes ejercicios de código en C++.

#### 2. Explicación del vector de interrupciones y los tipos de interrupciones


Partiendo de que una interrupción es una señal emitida por un programa o por un dispositivo de la computadora para que el CPU se detenga de hacer lo que hace y empiece a ejecutar las instrucciones que se almacenan en una direccion especifica de la memoria, dicha dirección especifica de memoria es proveida por el vector de interrupciones. Analizando un poco como seria el flujo de una interrupcion quedaria de la siguiente forma: Primer punto, el CPU esta trabajando en un proceso X; segundo punto, un dispositivo recibe un input entonces envia una señal o una interrupción al cpu con la intención de ser atendido; tercer punto, el CPU realiza un cambio de contexto, almacenando las direcciones y registros del proceso X para luego ser restablecido; cuarto punto , la interrupcion contiene parametros para indicar que interrupcion se requiere, en base a estos parametros es posible buscar la dirección de la rutina de ejecucion de dicha interrupcion en un vector que contiene las direcciones de la primera instrucción a ser ejecutada por la rutina de cada interrupción.


Las Interrupciones pueden ser de 3 tipos:
* Interrupciones de Hardware

    Estas interrupciones no se san por ninguna instruccion de algun programa que se esta ejecutando, sino se dan exclusivamente por los dispositivos de entrada o salida como por ejemplo el teclado, el mouse, entre otros.
    

* Excepciones

    Estas son interrupciones que se generan por un error, normalmente por operaciones invalidas o por el acceso a segmentos de memoria no permitidos. Esta interrupcion genera un cambio de contexto a los procesos del sistema operativo para que este maneje el error.
    
    
* Interrupciones por Software

    Estos son los famosos syscalls o system calls, dichas interrupciones son generadas por un programa en tiempo de ejecucion. Algunos ejemplos de estas interrupciones son cuando un programa requiere un servicio del sistema operativo para escribir sobre un archivo o cuando el programa necesita obtener la hora de la maquina.


#### Ejemplos utilizando interrupciones en ASM

A continuación se muestras dos ejemplos en ASM los cuales nos permitiran ver el uso de las interrupciones, en este caso por software ya que el programa las esta emitiendo.

* Ejemplo 1: haremos uso de la interrupción 21h para mostrar en pantalla un caracter dado su codigo ASCII.

In [ ]:
#include <dos.h>
#include <conio.h>
void main(){
    clrscr();
    asm{
        mov ah, 0x2 //se le asigna el valor de 2 al registro AH para indicar que usaremos la funcion de display
        mov dl,0x41 //se le asigna el codigo ASCII en hexadecimal al registro dl, que es por default donde ira a traerlo la funcion
        INT 0x21 //se hace la interrupcion 21
        
    }
}

* Ejemplo 2: por medio de interrupciones solicitaremos al usuario un digito entre 1 y 8, e imprimiremos en pantalla "numero + 1= resultado", osea le sumaremos el numero 1 al digito que nos ingrese el usuario. 

In [ ]:
#include <dos.h>
#include <conio.h>
void main(){
    clrscr();
    asm{
        mov ah, 0x1 //asginamos el 1 en ah, para indicar que queremos la funcion numero 1 de la interrupcion 21h, la cual solicita input
        INT 0x21 // realizamos la interrupcion para obtener el input
        
        mov ch, al // el input se guardara en al, entonces movemos a ch, el input 
        
        mov ah, 0x2 // asinamos el 2 en ah, para indicar la funcion 2 de la interrupcion 21h, la cual imprime a pantalla
        mod dl, 0x2B // le asignamos el codigo ASCII en hexadecimal del sigmo "+"
        INT 0x21 // realizamos la interrupcion para imprimir
        
        mov dl, 0x31 //le asigmanos el codigo ASCII en hexadecimal del numero "1"
        INT 0x21 // realizamos la interrupcion para imprimir
        
        mov dl, 3D // le asignamos el codigo ASCII en hexadecimal del simbolo "="
        INT 0x21 // realizamos la interrupcion para imprimir
        
        add, ch,0x1 // le sumamos al registro que contiene nuestro input en 1
        mov dl, ch // modemos nuestro resultado a dl para ser impreso
        INT 0x21 // realizamos la interrupcion del resultado
    }
}

#### Explicación de la interrupción 21h

La interrupcion 21h funciona como una DOS API la cual provee de multiples funciones para las interrupciones primarias de tipo Software. COmo anteriormente mencionamos, estas interrupciones son las que los programas invocan al necesitar de un servicio.
A continuacion en la siguiente imagen se listan las funciones con sus respectivas descripciones. Para ejecutar cada funcion es necesario que el registro AH contenga el valor que se especifica en la tabla. 

![21H](21h.png)

Para mayor detalle acerca de cada funcion y de que retorna es mejor visitar el siguiente link: http://stanislavs.org/helppc/int_21.html

#### Usando el 21H, obtener el vector de interrupción e identificar CS:PC

